In [ ]:
pip install ultralytics

pip install supervision

In [ ]:
from ultralytics import YOLO
import cv2
import argparse
import supervision as sv
import numpy as np

In [ ]:
ZONE_POLYGON = np.array([
    [0, 0],
    [0.5, 0],
    [0, 1],
    [0.5, 1]
])

In [ ]:
def parse_arguments() -> argparse.Namespace:
  parser = argparse.ArgumentParser(description = 'yolov8')
  parser.add_argument(
      '--webcam-resolution--',
      default = [1280, 720],
      nargs = 2,
      type = int
  )
  args = parser.parse_args()
  return args

In [ ]:
cap = cv2.videoCapture(0)
args = parse_arguments()
width, height = args.webcam_resolution
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

model = YOLO('yolov8l.pt')

box_annotator = sv.BoxAnnotator(
    thickness = 2,
    text_thickness = 2,
    text_scale = 1
)

zone_polygon = (ZONE_POLYGON * np.array(args.webcam_resolution)).astype(int)
zone = sv.PolygonZone(polygon = zone_polygon, frame_resolution_wh = tuple(args.webcam_resolution))
zone_annotator = sv.PolygonZoneAnnotator(
    zone = zone, 
    color = sv.Color.red(),
    thickness = 2,
    text_thickness = 4,
    text_scale = 2
)

while(True):
  ret, frame = cap.read()
  
  result = model(frame, agnostic_nms = True)[0]
  detections = sv.Detections.from_yolov8(result)
  detections = detections[detections.class_id == 47]
  labels = [
      f'{model.model.names[class_id]} {confidence:0.2f}'
      for _, confidence, class_id in detections
  ]
  frame = box_annotator.annotate(scene = frame, detections = detections, labels = labels)
  zone.trigger(detections = detections)
  frame = zone.annotate(scene = frame)
  cv2.imshow('yolov8', frame)
  if (cv2.waitKey(30) == 27):
    break
  